<a href="https://colab.research.google.com/github/jmontero23/SenalesySistemas/blob/main/Proyecto_Final/Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit -q
!pip install pyngrok -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.4 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import firwin, lfilter, hilbert, welch

# --- CONFIGURACIÓN DE LA PÁGINA ---
st.set_page_config(
    page_title="De Fourier al WiFi/5G",
    page_icon="📡",
    layout="wide",
    initial_sidebar_state="expanded"
)

# --- FUNCIONES AUXILIARES ---

def plot_signal(t, signal, title, xlabel="Tiempo (s)", ylabel="Amplitud"):
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(t, signal)
    ax.set_title(title, fontsize=16)
    ax.set_xlabel(xlabel, fontsize=12)
    ax.set_ylabel(ylabel, fontsize=12)
    ax.grid(True)
    ax.margins(x=0)
    st.pyplot(fig)

def plot_spectrum(signal, fs, title, nperseg=1024):
    fig, ax = plt.subplots(figsize=(10, 4))
    f, Pxx = welch(signal, fs, nperseg=nperseg)
    ax.semilogy(f, Pxx)
    ax.set_title(title, fontsize=16)
    ax.set_xlabel("Frecuencia (Hz)", fontsize=12)
    ax.set_ylabel("Densidad Espectral de Potencia (V^2/Hz)", fontsize=12)
    ax.grid(True)
    ax.margins(x=0)
    st.pyplot(fig)

def plot_constellation(symbols, title, order):
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.scatter(np.real(symbols), np.imag(symbols), alpha=0.5, edgecolors='none')
    ax.set_title(title, fontsize=16)
    ax.set_xlabel("En Fase (I)", fontsize=12)
    ax.set_ylabel("Cuadratura (Q)", fontsize=12)
    ax.grid(True)
    ax.axhline(0, color='black', lw=0.5)
    ax.axvline(0, color='black', lw=0.5)
    ref_symbols = get_qam_ref_symbols(order)
    lim_val = np.max(np.abs(ref_symbols)) * 1.5
    ax.set_xlim(-lim_val, lim_val)
    ax.set_ylim(-lim_val, lim_val)
    st.pyplot(fig)

def get_qam_symbols(N, order):
    bits_per_symbol = int(np.log2(order))
    num_bits = N * bits_per_symbol
    random_bits = np.random.randint(0, 2, num_bits)

    symbols = []
    if order == 4:
        for i in range(0, num_bits, 2):
            I = 2 * random_bits[i] - 1
            Q = 2 * random_bits[i+1] - 1
            symbols.append(I + 1j*Q)
    elif order == 16:
        for i in range(0, num_bits, 4):
            I = 2 * (random_bits[i] * 2 + random_bits[i+1]) - 3
            Q = 2 * (random_bits[i+2] * 2 + random_bits[i+3]) - 3
            symbols.append(I + 1j*Q)

    ref_symbols = get_qam_ref_symbols(order)
    norm_factor = np.sqrt(np.mean(np.abs(ref_symbols)**2))
    return np.array(symbols) / norm_factor

def get_qam_ref_symbols(order):
    if order == 4:
        return np.array([i + 1j*q for i in [-1, 1] for q in [-1, 1]])
    elif order == 16:
        return np.array([i + 1j*q for i in [-3, -1, 1, 3] for q in [-3, -1, 1, 3]])
    return np.array([])


# --- BARRA LATERAL DE CONTROLES ---
st.sidebar.title("🛠️ Parámetros de Simulación")
st.sidebar.markdown("Ajuste los parámetros para ver cómo afectan a la señal en cada etapa.")

st.sidebar.header("Fase 1: Dominio de la Frecuencia")
fs = st.sidebar.slider("Frecuencia de Muestreo (Hz)", 1000, 10000, 5000)
f0 = st.sidebar.slider("Frecuencia de la Señal (Hz)", 10, fs//10, 100)
cutoff_freq = st.sidebar.slider("Frecuencia de Corte del Filtro (Hz)", 10, fs//4, 250)

st.sidebar.header("Fase 3 y 4: Modulación QAM")
qam_order = st.sidebar.select_slider("Orden de QAM", options=[4, 16], value=16)
carrier_freq = st.sidebar.slider("Frecuencia Portadora (Hz)", fs//8, fs//2 -100, 1000)
snr_db = st.sidebar.slider("Relación Señal-Ruido (SNR) (dB)", -5, 30, 15)

# --- DEFINICIÓN DE PARÁMETROS DE TIEMPO ---
num_symbols = 256
samples_per_symbol = 20
N = num_symbols * samples_per_symbol
duration = N / fs
t = np.linspace(0, duration, N, endpoint=False)
numtaps_filter = 101 # Orden para todos los filtros FIR


# --- CUERPO PRINCIPAL DEL DASHBOARD ---
st.title("📡 De Fourier al WiFi/5G: Anatomía de una Señal Inalámbrica")
st.markdown("Este dashboard interactivo descompone los principios de una transmisión inalámbrica moderna, tal como se describe en el proyecto final del curso. Aquí puedes visualizar y experimentar con los conceptos clave, desde el análisis espectral hasta la modulación y demodulación QAM.")

# --- FASE 1: EL DOMINIO DE LA FRECUENCIA ---
st.header("Fase 1: El Dominio de la Frecuencia 📊")
st.markdown("Generamos una señal, la analizamos con la FFT para ver su contenido en frecuencia y luego la filtramos para eliminar componentes no deseadas.")
col1, col2 = st.columns(2)
with col1:
    st.subheader("1.1. Señal Original y su Espectro")
    signal = np.cos(2 * np.pi * f0 * t) + 0.5 * np.sin(2 * np.pi * f0 * 3.5 * t)
    plot_signal(t[:int(fs*0.05)], signal[:int(fs*0.05)], f"Señal Sintética en Tiempo (f₀ = {f0} Hz)")
with col2:
    st.subheader("1.2. Aplicando la FFT")
    plot_spectrum(signal, fs, "Espectro de la Señal Original")
st.subheader("1.3. Diseño y Aplicación de Filtro Paso-Bajo")
fir_coeff = firwin(numtaps_filter, cutoff_freq, fs=fs, window='hamming', pass_zero='lowpass')
filtered_signal = lfilter(fir_coeff, 1.0, signal)
col3, col4 = st.columns(2)
with col3:
    plot_signal(t[:int(fs*0.05)], filtered_signal[:int(fs*0.05)], f"Señal Filtrada (Corte a {cutoff_freq} Hz)")
with col4:
    plot_spectrum(filtered_signal, fs, "Espectro de la Señal Filtrada")

# --- FASE 2: CONSTRUYENDO LAS SEÑALES I/Q ---
st.header("Fase 2: Construyendo las Señales I/Q (Fase y Cuadratura) 🧬")
st.markdown("Usamos la Transformada de Hilbert sobre una señal mensaje (I) para generar su componente en cuadratura (Q), que está desfasada 90°.")
analytic_signal = hilbert(filtered_signal)
signal_I = np.real(analytic_signal)
signal_Q = np.imag(analytic_signal)
col5, col6 = st.columns(2)
with col5:
    st.subheader("Señal en Fase (I) vs. Cuadratura (Q)")
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(t[:200], signal_I[:200], label="Señal en Fase (I)")
    ax.plot(t[:200], signal_Q[:200], label="Señal en Cuadratura (Q)", linestyle='--')
    ax.set_title("Comparación de I(t) y Q(t) mostrando desfase", fontsize=16)
    ax.set_xlabel("Tiempo (s)", fontsize=12)
    ax.set_ylabel("Amplitud", fontsize=12)
    ax.legend()
    ax.grid(True)
    ax.margins(x=0)
    st.pyplot(fig)
with col6:
    st.subheader("Espectro de la Señal Analítica")
    plot_spectrum(analytic_signal, fs, "Espectro de la Señal Analítica (Hilbert)")

# --- FASE 3: MODULACIÓN QAM ---
st.header("Fase 3: Modulación de Amplitud en Cuadratura (QAM) 🛰️")
st.markdown("Mapeamos bits de información a símbolos complejos (I/Q) y los modulamos sobre una portadora para ser enviados por el aire.")
symbols = get_qam_symbols(num_symbols, qam_order)
ref_symbols = get_qam_ref_symbols(qam_order)
col7, col8 = st.columns([1, 2])
with col7:
    st.subheader(f"Constelación {qam_order}-QAM Ideal")
    plot_constellation(ref_symbols, f"Diagrama de {qam_order}-QAM Ideal", qam_order)
with col8:
    st.subheader("Señal Modulada QAM")
    symbols_upsampled = np.zeros(N, dtype=complex)
    symbols_upsampled[::samples_per_symbol] = symbols
    pulse = np.ones(samples_per_symbol)
    baseband_signal = lfilter(pulse, 1.0, symbols_upsampled)
    carrier_signal = np.exp(1j * 2 * np.pi * carrier_freq * t)
    qam_modulated_signal = np.real(baseband_signal * carrier_signal)
    plot_signal(t[:400], qam_modulated_signal[:400], f"Señal Modulada ({qam_order}-QAM) en el Tiempo")
    plot_spectrum(qam_modulated_signal, fs, f"Espectro de la Señal Modulada ({qam_order}-QAM)")

# --- FASE 4: EL SISTEMA COMPLETO Y CORREGIDO ---
st.header("Fase 4: Sistema Completo con Ruido y Demodulación 📡➡️💻")
st.markdown("Simulamos un canal de comunicación añadiendo Ruido Gaussiano Blanco Aditivo (AWGN) y observamos cómo afecta a la constelación en el receptor.")
signal_power = np.mean(np.abs(qam_modulated_signal)**2)
snr_linear = 10**(snr_db / 10)
noise_power = signal_power / snr_linear
noise = np.random.normal(0, np.sqrt(noise_power), N)
received_signal = qam_modulated_signal + noise

# --- LÓGICA DE DEMODULACIÓN CORREGIDA ---
# 1. Bajar la señal a banda base
demod_signal_raw = received_signal * 2 * np.exp(-1j * 2 * np.pi * carrier_freq * t)

# 2. Aplicar filtro paso-bajo para eliminar armónicos
demod_filter_cutoff = fs / samples_per_symbol
fir_coeff_demod = firwin(numtaps_filter, cutoff=demod_filter_cutoff, fs=fs)
demod_signal_filtered = lfilter(fir_coeff_demod, 1.0, demod_signal_raw)

# 3. **COMPENSAR EL RETARDO DEL FILTRO Y MUESTREAR **
delay = (numtaps_filter - 1) // 2
# Creamos los instantes de muestreo, comenzando a la mitad del primer símbolo
sampling_instants = np.arange(samples_per_symbol//2, N, samples_per_symbol)
# Ajustamos los instantes para compensar el retardo del filtro
sampling_instants_delayed = sampling_instants - delay
# Tomamos solo los instantes válidos (mayores o iguales a cero)
valid_instants = sampling_instants_delayed[sampling_instants_delayed >= 0].astype(int)
received_symbols = demod_signal_filtered[valid_instants]


col9, col10 = st.columns(2)
with col9:
    st.subheader("Señal Recibida con Ruido")
    plot_signal(t[:400], received_signal[:400], f"Señal Recibida (SNR = {snr_db} dB)")
with col10:
    st.subheader("Constelación en el Receptor (Corregida)")
    plot_constellation(received_symbols, f"Constelación Recibida (SNR = {snr_db} dB)", qam_order)

# --- FIN ---
st.info("Proyecto Final - Señales y Sistemas 2025")

Overwriting app.py


In [ ]:
# Pega tu authtoken de ngrok aquí
!ngrok config add-authtoken '2zLBw00gDbYx9vnFZneHVhwzE62_UptNrhkYSDvxyYsuhwng'

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

# Ejecutar streamlit en segundo plano y silenciar su salida
!streamlit run app.py &> /dev/null &

# Crear el túnel con ngrok e imprimir la URL pública
public_url = ngrok.connect(8501)
print(f"Haga clic en este enlace para abrir el dashboard: {public_url}")

Haga clic en este enlace para abrir el dashboard: NgrokTunnel: "https://fba58a066907.ngrok-free.app" -> "http://localhost:8501"
